In [1]:
import scipy
import scipy.linalg as linalg
import numpy as np
!pip install igraph
import igraph as ig

In [46]:
class NOTEARS:
  def __init__(self, weight_threshold, max_rho, max_iter, h_tol, lambda1):
    self.X = None
    self.weight_threshold = weight_threshold
    self.max_rho = max_rho
    self.max_iter = max_iter
    self.h_tol = h_tol
    self.lambda1 = lambda1
    self.rho = 1.0
    self.alpha = 0.0

  def h(self, W_flattened):
    d = self.d
    W = W_flattened.reshape([d, d])
    ###W = self._adj(W_flattened)
    M = linalg.expm(W*W)
    h = np.trace(M) - d
    h_grad = 2* M.T * W
    return h, h_grad


  # Doesn't include regularization term
  def objective_and_gradient(self, W_flattened):
      d = self.X.shape[1]
      n = self.X.shape[0]

      W = W_flattened.reshape([d, d])
      ###W = self._adj(W_flattened)

      # Just lasso regression loss
      R = self.X - self.X @ W
      loss = 1/(2*n) * (R**2).sum()
      grad_loss = -1.0/n * self.X.T @ R
      h, h_grad = self.h(W_flattened)
      objective_value = loss + self.rho/2 * h**2 + self.alpha*h
      objective_gradient = grad_loss + (self.rho * h + self.alpha) * h_grad

      ###objective_value = loss + self.rho/2 * h**2 + self.alpha*h + self.lambda1*np.abs(W).sum()
      ###objective_gradient = grad_loss + (self.rho * h + self.alpha) * h_grad
      ###objective_gradient = np.concatenate((objective_gradient + self.lambda1, - objective_gradient + self.lambda1), axis=None)
      return objective_value, objective_gradient.flatten()

  def _adj(self, w):
        """Convert doubled variables ([2 d^2] array) back to original variables ([d, d] matrix)."""
        return (w[:d * d] - w[d * d:]).reshape([d, d])


  def solve(self):
      d = self.X.shape[1]
      n = self.X.shape[0]
      # Starting guess for W; initialize rho, alpha
      W_est_flattened, self.rho, self.alpha, h = np.zeros(d*d), 1.0, 0.0, np.inf

      # Set bounds for W_est matrix (basically diagonal elements must be zero.)
      bnds = [(0, 0) if i == j else (0, None) for i in range(d) for j in range(d)]
      ###bnds = [(0, 0) if i == j else (0, None) for _ in range(2) for i in range(d) for j in range(d)]

      # Perform Dual Gradient ascent to solve eq. (12) in paper
      for i in range(1, self.max_iter, 1):
          print("Iteration " + str(i))

          while self.rho < self.max_rho:
              W_next_flattened = scipy.optimize.minimize(self.objective_and_gradient, W_est_flattened, method='L-BFGS-B', jac=True, bounds = bnds).x
              #print(W_next_flattened)
              h_next, _ = self.h(W_next_flattened)
              #print(h_next)
              if h_next > 0.25*h: self.rho*= 10
              else: break

          W_est_flattened = W_next_flattened
          h = h_next
          self.alpha = self.alpha + self.rho*h
          print(h)

          if h <= self.h_tol or self.rho >= self.max_rho: break

      # Threshold values lower than weight_threshold (omega)
      W_est = W_est_flattened.reshape([d, d])
      ###W = self._adj(W_est_flattened)
      W_est[np.abs(W_est)< self.weight_threshold] = 0
      assert self.is_dag(W_est)
      np.savetxt('W_est.csv', W_est, delimiter=',')


      accuracy = self.count_accuracy(B_true, W_est != 0)
      print(accuracy)
      return W_est


  ### Helper functions to generate/simulate DAG (i.e. our synthetic data) (copied from NOTEARS code)
  def is_dag(self, W):
    G = ig.Graph.Weighted_Adjacency(W.tolist())
    return G.is_dag()

  def simulate_parameter(self, B, w_ranges=((-2.0, -0.5), (0.5, 2.0))):
    """Simulate SEM parameters for a DAG.

    Args:
        B (np.ndarray): [d, d] binary adj matrix of DAG
        w_ranges (tuple): disjoint weight ranges

    Returns:
        W (np.ndarray): [d, d] weighted adj matrix of DAG
    """
    W = np.zeros(B.shape)
    S = np.random.randint(len(w_ranges), size=B.shape)  # which range
    for i, (low, high) in enumerate(w_ranges):
        U = np.random.uniform(low=low, high=high, size=B.shape)
        W += B * (S == i) * U
    return W


  def simulate_dag(self, d, s0, graph_type):
      """Simulate random DAG with some expected number of edges.

      Args:
          d (int): num of nodes
          s0 (int): expected num of edges
          graph_type (str): ER, SF, BP

      Returns:
          B (np.ndarray): [d, d] binary adj matrix of DAG
      """
      def _random_permutation(M):
          # np.random.permutation permutes first axis only
          P = np.random.permutation(np.eye(M.shape[0]))
          return P.T @ M @ P

      def _random_acyclic_orientation(B_und):
          return np.tril(_random_permutation(B_und), k=-1)

      def _graph_to_adjmat(G):
          return np.array(G.get_adjacency().data)

      if graph_type == 'ER':
          # Erdos-Renyi
          G_und = ig.Graph.Erdos_Renyi(n=d, m=s0)
          B_und = _graph_to_adjmat(G_und)
          B = _random_acyclic_orientation(B_und)
      elif graph_type == 'SF':
          # Scale-free, Barabasi-Albert
          G = ig.Graph.Barabasi(n=d, m=int(round(s0 / d)), directed=True)
          B = _graph_to_adjmat(G)
      elif graph_type == 'BP':
          # Bipartite, Sec 4.1 of (Gu, Fu, Zhou, 2018)
          top = int(0.2 * d)
          G = ig.Graph.Random_Bipartite(top, d - top, m=s0, directed=True, neimode=ig.OUT)
          B = _graph_to_adjmat(G)
      else:
          raise ValueError('unknown graph type')
      B_perm = _random_permutation(B)
      assert ig.Graph.Adjacency(B_perm.tolist()).is_dag()
      return B_perm

  def simulate_linear_sem(self, W, n, sem_type, noise_scale=None):
      """Simulate samples from linear SEM with specified type of noise.

      For uniform, noise z ~ uniform(-a, a), where a = noise_scale.

      Args:
          W (np.ndarray): [d, d] weighted adj matrix of DAG
          n (int): num of samples, n=inf mimics population risk
          sem_type (str): gauss, exp, gumbel, uniform, logistic, poisson
          noise_scale (np.ndarray): scale parameter of additive noise, default all ones

      Returns:
          X (np.ndarray): [n, d] sample matrix, [d, d] if n=inf
      """
      def _simulate_single_equation(X, w, scale):
          """X: [n, num of parents], w: [num of parents], x: [n]"""
          if sem_type == 'gauss':
              z = np.random.normal(scale=scale, size=n)
              x = X @ w + z
          elif sem_type == 'exp':
              z = np.random.exponential(scale=scale, size=n)
              x = X @ w + z
          elif sem_type == 'gumbel':
              z = np.random.gumbel(scale=scale, size=n)
              x = X @ w + z
          elif sem_type == 'uniform':
              z = np.random.uniform(low=-scale, high=scale, size=n)
              x = X @ w + z
          elif sem_type == 'logistic':
              x = np.random.binomial(1, scipy.special.expit(X @ w)) * 1.0
          elif sem_type == 'poisson':
              x = np.random.poisson(np.exp(X @ w)) * 1.0
          else:
              raise ValueError('unknown sem type')
          return x

      d = W.shape[0]
      if noise_scale is None:
          scale_vec = np.ones(d)
      elif np.isscalar(noise_scale):
          scale_vec = noise_scale * np.ones(d)
      else:
          if len(noise_scale) != d:
              raise ValueError('noise scale must be a scalar or has length d')
          scale_vec = noise_scale
      if not self.is_dag(W):
          raise ValueError('W must be a DAG')
      if np.isinf(n):  # population risk for linear gauss SEM
          if sem_type == 'gauss':
              # make 1/d X'X = true cov
              X = np.sqrt(d) * np.diag(scale_vec) @ np.linalg.inv(np.eye(d) - W)
              return X
          else:
              raise ValueError('population risk not available')
      # empirical risk
      G = ig.Graph.Weighted_Adjacency(W.tolist())
      ordered_vertices = G.topological_sorting()
      assert len(ordered_vertices) == d
      X = np.zeros([n, d])
      for j in ordered_vertices:
          parents = G.neighbors(j, mode=ig.IN)
          X[:, j] = _simulate_single_equation(X[:, parents], W[parents, j], scale_vec[j])
      return X

  def count_accuracy(self, B_true, B_est):
    """Compute various accuracy metrics for B_est.

    true positive = predicted association exists in condition in correct direction
    reverse = predicted association exists in condition in opposite direction
    false positive = predicted association does not exist in condition

    Args:
        B_true (np.ndarray): [d, d] ground truth graph, {0, 1}
        B_est (np.ndarray): [d, d] estimate, {0, 1, -1}, -1 is undirected edge in CPDAG

    Returns:
        fdr: (reverse + false positive) / prediction positive
        tpr: (true positive) / condition positive
        fpr: (reverse + false positive) / condition negative
        shd: undirected extra + undirected missing + reverse
        nnz: prediction positive
    """
    if (B_est == -1).any():  # cpdag
        if not ((B_est == 0) | (B_est == 1) | (B_est == -1)).all():
            raise ValueError('B_est should take value in {0,1,-1}')
        if ((B_est == -1) & (B_est.T == -1)).any():
            raise ValueError('undirected edge should only appear once')
    else:  # dag
        if not ((B_est == 0) | (B_est == 1)).all():
            raise ValueError('B_est should take value in {0,1}')
        if not self.is_dag(B_est):
            raise ValueError('B_est should be a DAG')
    d = B_true.shape[0]
    # linear index of nonzeros
    pred_und = np.flatnonzero(B_est == -1)
    pred = np.flatnonzero(B_est == 1)
    cond = np.flatnonzero(B_true)
    cond_reversed = np.flatnonzero(B_true.T)
    cond_skeleton = np.concatenate([cond, cond_reversed])
    # true pos
    true_pos = np.intersect1d(pred, cond, assume_unique=True)
    # treat undirected edge favorably
    true_pos_und = np.intersect1d(pred_und, cond_skeleton, assume_unique=True)
    true_pos = np.concatenate([true_pos, true_pos_und])
    # false pos
    false_pos = np.setdiff1d(pred, cond_skeleton, assume_unique=True)
    false_pos_und = np.setdiff1d(pred_und, cond_skeleton, assume_unique=True)
    false_pos = np.concatenate([false_pos, false_pos_und])
    # reverse
    extra = np.setdiff1d(pred, cond, assume_unique=True)
    reverse = np.intersect1d(extra, cond_reversed, assume_unique=True)
    # compute ratio
    pred_size = len(pred) + len(pred_und)
    cond_neg_size = 0.5 * d * (d - 1) - len(cond)
    fdr = float(len(reverse) + len(false_pos)) / max(pred_size, 1)
    tpr = float(len(true_pos)) / max(len(cond), 1)
    fpr = float(len(reverse) + len(false_pos)) / max(cond_neg_size, 1)
    # structural hamming distance
    pred_lower = np.flatnonzero(np.tril(B_est + B_est.T))
    cond_lower = np.flatnonzero(np.tril(B_true + B_true.T))
    extra_lower = np.setdiff1d(pred_lower, cond_lower, assume_unique=True)
    missing_lower = np.setdiff1d(cond_lower, pred_lower, assume_unique=True)
    shd = len(extra_lower) + len(missing_lower) + len(reverse)
    return {'fdr': fdr, 'tpr': tpr, 'fpr': fpr, 'shd': shd, 'nnz': pred_size}

In [47]:
solver = NOTEARS(weight_threshold= 0.3, max_rho= 1e16, max_iter= 100, h_tol= 1e-8, lambda1= 0.0)
n, d, s0, graph_type, sem_type = 100, 20, 20, 'ER', 'gauss'
solver.d = d
B_true = solver.simulate_dag(d, s0, graph_type)
W_true = solver.simulate_parameter(B_true)
np.savetxt('W_true.csv', W_true, delimiter=',')

solver.X = solver.simulate_linear_sem(W_true, n, sem_type)
np.savetxt('X.csv', solver.X, delimiter=',')

In [48]:
%%time
W_est = solver.solve()

Iteration 1
0.9350416285990306
Iteration 2
0.10722166556197621
Iteration 3
0.02603735740373736
Iteration 4
0.006007220596369933
Iteration 5
0.0013180540243773464
Iteration 6
0.00023685313700383404
Iteration 7
5.406960038456532e-05
Iteration 8
1.1905087365704503e-05
Iteration 9
2.6535196226973312e-06
Iteration 10
5.492156560649164e-07
Iteration 11
1.1979870251366265e-07
Iteration 12
2.613833771647478e-08
Iteration 13
5.698520766372894e-09
{'fdr': 0.7352941176470589, 'tpr': 0.45, 'fpr': 0.14705882352941177, 'shd': 36, 'nnz': 34}
CPU times: user 4.66 s, sys: 3.6 s, total: 8.25 s
Wall time: 4.4 s


In [49]:
solver.rho

10000000000000.0

In [50]:
solver.alpha

105446.72186632639